In [6]:
from PIL import Image

# Open the RGB image
image = Image.open('Hibiscus_color.png')

# Split the image into RGB channels
red_channel, green_channel, blue_channel = image.split()

# Save each channel as a separate image
red_channel.save('red_channel.png')
green_channel.save('green_channel.png')
blue_channel.save('blue_channel.png')



In [7]:

from PIL import Image
import math

# Open the RGB image
image = Image.open('Hibiscus_color.png')

# Separate the image into its RGB channels
red_channel, green_channel, blue_channel = image.split()

# Create blank images for HSL channels
h_channel = Image.new('L', image.size)
s_channel = Image.new('L', image.size)
l_channel = Image.new('L', image.size)

for x in range(image.width):
    for y in range(image.height):
        r = red_channel.getpixel((x, y)) / 255.0
        g = green_channel.getpixel((x, y)) / 255.0
        b = blue_channel.getpixel((x, y)) / 255.0

        cmax = max(r, g, b)
        cmin = min(r, g, b)
        delta = cmax - cmin



        # Calculate Hue (H)
        if delta == 0:
            h = 0
        elif cmax == r:
            h = 60 * (((g - b) / delta) % 6)
        elif cmax == g:
            h = 60 * (((b - r) / delta) + 2)
        else:
            h = 60 * (((r - g) / delta) + 4)
        h_channel.putpixel((x, y), int((h / 360) * 255))


        # Calculate Lightness (L)
        l = (cmax + cmin) / 2.0
        l_channel.putpixel((x, y), int(255 * l))


        # Calculate Saturation (S)
        if delta == 0:
            s = 0
        else:
            s = delta / (1 - abs(2 * l - 1))
        s_channel.putpixel((x, y), int(255 * s))


# Save each channel separately
h_channel.save('H_channel.png')
s_channel.save('S_channel.png')
l_channel.save('L_channel.png')

In [8]:

from PIL import Image
import math

# HSL channels
h_channel = Image.open('H_channel.png')
s_channel = Image.open('S_channel.png')
l_channel = Image.open('L_channel.png')

# Create a blank RGB image
rgb_image = Image.new('RGB', h_channel.size)

for x in range(h_channel.width):
    for y in range(h_channel.height):
        h = (h_channel.getpixel((x, y)) / 255.0) * 360.0
        s = s_channel.getpixel((x, y)) / 255.0
        l = l_channel.getpixel((x, y)) / 255.0

        # Calculate the HSL to RGB conversion
        c = (1 - abs(2 * l - 1)) * s
        X = c * (1 - abs((h / 60) % 2 - 1))
        m = l - c / 2

        r, g, b = 0, 0, 0

        if 0 <= h < 60:
            r, g, b = c, X, 0
        elif 60 <= h < 120:
            r, g, b = X, c, 0
        elif 120 <= h < 180:
            r, g, b = 0, c, X
        elif 180 <= h < 240:
            r, g, b = 0, X, c
        elif 240 <= h < 300:
            r, g, b = X, 0, c
        else:
            r, g, b = c, 0, X

        r = int((r + m) * 255)
        g = int((g + m) * 255)
        b = int((b + m) * 255)

        # Set the RGB pixel in the reconstructed image
        rgb_image.putpixel((x, y), (r, g, b))

# Save the reconstructed RGB image
rgb_image.save('Reconstructed_RGB.png')
rgb_image.show()

In [9]:
from PIL import Image
import numpy as np

# Histogram equalization function
def histogram_equalization(channel):
    # Calculate the histogram of the image
    histogram, _ = np.histogram(channel, bins=256, range=(0, 256))

    # Calculate the cumulative distribution function (Computing the cumulative histogram)
    cdf = histogram.cumsum()

    # Perform histogram equalization
    equalized_pixels = np.uint8(cdf / (width * height) * 255)

    return Image.fromarray(equalized_pixels[channel])


# Open the grayscale image
image = Image.open('Hibiscus.png').convert('L')
width, height = image.size

# Create an image with the equalized pixels
equalized_image = histogram_equalization(image)

# Save or display the equalized image
equalized_image.save('equalized_Hibiscus.png')
equalized_image.show()


# Load the R, G, and B channels as grayscale images
red_channel = Image.open('red_channel.png').convert('L')
green_channel = Image.open('green_channel.png').convert('L')
blue_channel = Image.open('blue_channel.png').convert('L')

# Calculate the width and height
width, height = red_channel.size

# Apply histogram equalization to each channel
red_equalized = histogram_equalization(red_channel)
green_equalized = histogram_equalization(green_channel)
blue_equalized = histogram_equalization(blue_channel)

# Merge the equalized channels into an RGB image
equalized_image = Image.merge("RGB", (red_equalized, green_equalized, blue_equalized))

# Save the equalized RGB image
equalized_image.save('rgb_equalized.png')
equalized_image.show()

# Load the L, H, and S channels as grayscale images
l_channel = Image.open('L_channel.png').convert('L')
h_channel = Image.open('H_channel.png').convert('L')
s_channel = Image.open('S_channel.png').convert('L')

# Calculate the width and height
width, height = l_channel.size

# Apply histogram equalization to the L channel
l_equalized = histogram_equalization(l_channel)

# Reconstruct the image with the new L channel and the original H and S channels
rgb_equalized = Image.merge("RGB", (h_channel, s_channel, l_equalized))

# Convert the result to an RGB image
#rgb_equalized = hsl_equalized.convert("RGB")

# Save the equalized HSL image as "HSL_equalized.png"
rgb_equalized.save('HSL_equalized.png')
rgb_equalized.show()
